In [1]:
import pandas as pd
import geopandas as gpd
import folium
import branca.colormap as cm
import branca

print(folium.__version__)

ModuleNotFoundError: No module named 'geopandas'

# Interactive Choropleth Map using Folium and Python

Cudos to:

Medium: https://medium.com/@lindsayrallen1/interactive-choropleth-map-of-washington-dc-using-folium-and-python-2794708514d5

Github: https://github.com/lindsallen/mapping/blob/master/dc_choropleth_wtooltip.ipynb

### Folium.docs 
    https://python-visualization.github.io/folium/index.html

First part taken from: 

### Interactive choropleth with Python and Folium (and some tips)

https://vverde.github.io/blob/interactivechoropleth.html

In [ ]:
# Read geodata nrw_gemeinden from data folder

fname = './data/nrw_gemeinden.json'
map = gpd.read_file(fname)
map.head()

In [ ]:
map=map[['AGS', 'GEN', 'geometry']]
map.head()

In [ ]:
map.crs = "EPSG:4326"

In [ ]:
population = pd.read_csv('./data/durchschnittsalter_nrw_gemeinden.csv', dtype={'AGS': 'str'}, decimal=',')
population.head(5)

In [ ]:
population=population[['AGS','alter']]
population.head(5)

In [ ]:
mappop=map.merge(population,on="AGS")
mappop.head()

In [ ]:
mappop.columns

In [ ]:
x_map=map.centroid.x.mean()
y_map=map.centroid.y.mean()
print(x_map,y_map)

In [ ]:
m = folium.Map(location=[y_map, x_map], zoom_start=8,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)
m

### folium.Choropleth

### folium.GeoJSON

In [ ]:
variable = 'alter'
name = 'Durchschnittsalter'
print(name, 'colorscale')

print('Min:', mappop.alter.min())
print('Max:', mappop.alter.max())

In [ ]:
df = mappop[['GEN', variable]].sort_values(by=variable, ascending=False)
df.reset_index(inplace=True)
quantiles = list(df[df.index.isin([0,66, 132, 198, 264, 330])][variable])
quantiles.append(0)
quantiles.sort()
quantiles

In [ ]:
colorscale = branca.colormap.linear.YlOrRd_09.scale(mappop.alter.min(), mappop.alter.max())
colorscale = colorscale.to_step(n=6, quantiles=quantiles)
colorscale.caption = name ## adds name for legend
colorscale

In [ ]:
folium.GeoJson(mappop, ## GeoPandas dataframe
               name="Durchschnittsalter",
                   
               ## controls the fill of the geo regions; applying colorscale based on variable
               style_function=lambda x: {"weight":1
                                         , 'color': '#545453'
                                         ## if variable is 0 map is a very light grey
                                         ## else colorscale applies based on variable
                                         , 'fillColor':'#9B9B9B' if x['properties'][variable] == 0 
                                         else colorscale(x['properties'][variable])
                                         ## similarly opacity is increased if value is 0
                                         , 'fillOpacity': 0.2 if x['properties'][variable] == 0 
                                         else 0.5},
                   
               ## changes styling of geo regions upon hover
               highlight_function=lambda x: {'weight':3, 'color':'black', 'fillOpacity': 1}, 
               
                ## tooltip can include information from any column in the GeoPandas dataframe   
                tooltip=folium.features.GeoJsonTooltip(
                fields=['GEN', variable],
                aliases=['Ort:', name])
              ).add_to(m)

## add colorscale to map so that it appears as the legend
colorscale.add_to(m)
    
m